In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity, SEIRHD_Bed

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import save_dict_and_create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Perform M0, M1 and M2 fits

In [ ]:
predictions_dict['m0'] = single_fitting_cycle(
    state, district, data_from_tracker=False, granular_data=True, filename=None, #Data
    model=SEIRHD, variable_param_ranges=None, #Choose Model and Ranges
    train_period=14, val_period=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['total', 'active', 'recovered', 'deceased'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'T_recov_hq': (50, 70),
    'T_recov_non_oxy': (5, 10),
    'T_recov_oxy': (15, 25),
    'T_recov_icu': (0, 20),
    'T_recov_vent': (0, 10),
    'P_non_oxy': (0, 0.15),
    'P_oxy': (0, 0.3),
    'P_icu': (0, 0.1),
    'P_vent': (0, 0.1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

In [ ]:
default_params = copy.copy(predictions_dict['m0']['best_params'])
del default_params['T_recov_severe']

predictions_dict['m1'] = single_fitting_cycle(
    state, district, data_from_tracker=False, granular_data=True, filename=None, #Data
    model=SEIRHD_Bed, variable_param_ranges=variable_param_ranges, default_params=default_params, #Choose Model and Ranges
    train_period=21, val_period=0, num_evals=1250, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['active', 'hq', 'icu', 'non_o2_beds', 'o2_beds', 'ventilator'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']

In [ ]:
predictions_dict['m1']['best_params']

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)